In [152]:
import pandas as pd
import numpy as np
from random import *

In [153]:
all_teams = pd.read_csv('teams.csv')
seasons= pd.read_csv('Seasons.csv')
reg_compact = pd.read_csv('RegularSeasonCompactResults.csv')
reg_detail = pd.read_csv('RegularSeasonDetailedResults.csv')
tour_compact = pd.read_csv('TourneyCompactResults.csv')
tour_detail = pd.read_csv('TourneyDetailedResults.csv')
tour_seeds = pd.read_csv('TourneySeeds.csv')
tour_slots = pd.read_csv('TourneySlots.csv')
sample_submission = pd.read_csv('SampleSubmission.csv')

I am going to create a function to score preductions for a specific year, based on the Kaggle Loss Score

In [154]:
##Takes a list of teams invited to tournament for a given year and returns a list of all possible matchups

def game_list(year):
    games = []
    teams = tour_seeds[tour_seeds.Season == year].Team.values
    for index, team1 in enumerate(teams):
        for team2 in teams[index+1:]:
            games.append(str(year) + '_' + str(team1) + '_' + str(team2))
    return games

In [155]:
submit_2003 = pd.DataFrame()
submit_2003['Id']= game_list(2003)
submit_2003['prob'] = [randint(1, 99)/100 for x in submit_2003['Id']]

In [156]:
## Takes a year and returns a list of results -- each result is a string in the form year_Wteam_Lteam 

def results_makelist(year):  
    resultlist = []
    results = tour_compact[tour_compact['Season'] == year]
    for index, row in results.iterrows():
        resultlist.append(str(row['Season']) + '_' + str(row['Wteam']) + '_' + str(row['Lteam']))
    return resultlist      




In [228]:
## takes a year and a team and provides the numeric portion of  such team for such year as integer

def get_seed(year, team):
    df_seeds = tour_seeds[tour_seeds.Season == year]
    
    try: 
        seed = df_seeds[df_seeds.Team == int(team)]['Seed'].values[0]
        return int(seed[1:3])
    
    except:
        return None
    

In [226]:
get_seed(1985, 1121)

In [229]:
get_seed(1985, 1116)

9

In [215]:
## takes a submission of predictions for a given year and scores the predictions for such year using the
## Kaggle formala for log loss.

def model_score(predictions = sample_submission):
        
    year = int(predictions.iloc[0,0][0:4])
    game_data = tour_compact[tour_compact['Season'] == year]
    results = results_makelist(year)
    predicts = fillout_submission(predictions)
    sigma = 0
    for result in results:
        sigma += np.log(predicts[result])
    return(0 - sigma/(len(results)))   

model_score(submit_2003)

1.0049025371862554

Create Function to Calculate Point Differential and Schedule Difficulty for each team for a given year

In [216]:
def make_factors(year):
    
    def difficulty(factors):
        strength_dic = dict(zip(factors.Team.values, factors.Diff.values))
        difficulty_schedule = []
        for team in factors['Team']:
            opponents = []
            for i in results.index:
                if results.loc[i, 'Wteam'] == team:
                    opponents.append(results.loc[i, 'Lteam'])
                elif results.loc[i, 'Lteam'] == team:
                    opponents.append(results.loc[i, 'Wteam'])
            opp_strength = 0
            for opponent in opponents:
                opp_strength += strength_dic[opponent]/len(opponents)
            difficulty_schedule.append(opp_strength)
        return difficulty_schedule
    
    
    
    team_list = [team for team in all_teams.iloc[:, 0]]
    dif_list = []
    results = reg_compact[reg_compact['Season'] == year]
    team_year_list = team_list.copy()
    for team in team_list:
        point_dif = 0
        games_played = 0
        for i in results.index:
            if results.loc[i, 'Wteam'] == team:
                point_dif = point_dif + results.loc[i, 'Wscore'] - results.loc[i, 'Lscore']
                games_played += 1
            elif results.loc[i, 'Lteam'] == team:
                point_dif = point_dif + results.loc[i, 'Lscore'] - results.loc[i, 'Wscore']
                games_played += 1
        if games_played == 0:
            team_year_list.remove(team)
        else:
            dif_list.append(point_dif/games_played)
    
    
        
    
    
    
    factors = pd.DataFrame()
    factors['Team'] = [team for team in team_year_list]
    factors['Diff'] = [diff for diff in dif_list]
    factors['schedule_str'] = [strength for strength in difficulty(factors)]
    factors['Seed'] = [get_seed(year, team) for team in team_year_list]
    return factors

In [217]:
factors 1985 = make_factors(1985)

,Team,Diff,schedule_str,Seed
0,1102,-5.791667,-0.324951,NaN
1,1103,-3.043478,-1.510071,NaN
2,1104,7.800000,1.091501,7.0
3,1106,-3.791667,-0.506431,NaN
4,1108,7.960000,-1.581587,NaN
5,1109,-29.125000,-0.343293,NaN
6,1110,-7.760000,0.813161,NaN
7,1111,-2.791667,-0.241291,NaN
8,1112,7.185185,0.486362,10.0
9,1113,-4.074074,2.048043,NaN


In [174]:
factors1985.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 3 columns):
Team            282 non-null int64
Diff            282 non-null float64
schedule_str    282 non-null float64
dtypes: float64(2), int64(1)
memory usage: 6.7 KB


In [266]:
results1985 = results_makelist(1985)


wteams1985 = [result[5:9] for result in results1985]
lteams1985 = [result[10:14] for result in results1985]
wdiffs1985 = [factors1985[factors1985.Team == int(team)]['Diff'] for team in wteams1985]
wsOs1985 = [factors1985[factors1985.Team == int(team)]['schedule_str'] for team in wteams1985]
ldiffs1985 = [factors1985[factors1985.Team == int(team)]['Diff'] for team in lteams1985]
lsOs1985 = [factors1985[factors1985.Team == int(team)]['schedule_str'] for team in lteams1985]

draft = pd.DataFrame()
draft['team1'] = wteams1985[0:31] + lteams1985[31:63]
draft['team2'] = lteams1985[0:31] + wteams1985[31:63]
draft['diffs1'] = wdiffs1985[0:31] + ldiffs1985[31:63]
draft['diffs2'] = ldiffs1985[0:31] + wdiffs1985[31:63]
draft['seed1'] = [get_seed(1985, team) for team in draft['team1']]
draft['seed2'] = [get_seed(1985, team) for team in draft['team2']]

In [267]:
draft.shape

(63, 6)